# Project 2: 911 Emergency Demand and Population Differences in NYC

This project explores how 911 emergency workloads differ across New York City boroughs, and whether boroughs with heavier emergency demand per capita experience longer average response times. I combine two NYC Open Data sources – 911 end-to-end response time data and population by borough – and use pandas to clean, merge, and visualize their relationship in a single plot.

Data Sources
This project uses two publicly available NYC Open Data sources:
911 End-to-End Emergency Response Time
https://data.cityofnewyork.us/Public-Safety/911-End-to-End-Data/t7p9-n9dy/about_data
NYC Population by Borough, 1950–2040
https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9/about_data
Both datasets are downloaded locally into my project folder as CSV files.

In [1]:
import pandas as pd
import plotly.express as px

print("✅ Project 2 notebook is ready.")

✅ Project 2 notebook is ready.


In [3]:
# Load the two datasets
calls = pd.read_csv("911 Open Data.csv")
pop   = pd.read_csv("Population Borough.csv")

calls.head(), calls.columns, pop.head(), pop.columns

(  Month Name     Agency                                        Description  \
 0  2025 / 09  Aggregate  Combined average response time to life threate...   
 1  2025 / 09  Aggregate  Combined average response time to life threate...   
 2  2025 / 09  Aggregate  Combined average response time to life threate...   
 3  2025 / 09  Aggregate  Combined average response time to life threate...   
 4  2025 / 09  Aggregate  Combined average response time to life threate...   
 
          Borough # of Incidents Response Times  
 0       Brooklyn         13,913         620.29  
 1          Bronx         11,423         894.03  
 2      Manhattan         11,883         786.83  
 3         Queens         10,060         694.58  
 4  Staten Island          2,178         589.96  ,
 Index(['Month Name', 'Agency', 'Description', 'Borough', '# of Incidents',
        'Response Times'],
       dtype='object'),
           Age Group       Borough       1950 1950 - Boro share of NYC total  \
 0  Total Popula

In [11]:
# Standardize column names
calls = calls.rename(columns={
    "Borough": "borough",
    "# of Incidents": "num_incidents",
    "Response Times": "response_seconds"
})

pop = pop.rename(columns={
    "Borough": "borough",
    "2020": "population"
})

# Keep only total population row
pop = pop[pop["Age Group"] == "Total Population"].copy()

# Convert numeric fields (remove commas)
calls["num_incidents"] = calls["num_incidents"].replace(",", "", regex=True).astype(float)
pop["population"] = pop["population"].replace(",", "", regex=True).astype(float)

# Standardize borough formatting
calls["borough"] = calls["borough"].str.strip().str.title()
pop["borough"]   = pop["borough"].str.strip().str.title()

calls.head(), pop.head()

# Convert response time to numeric
calls["response_seconds"] = (
    calls["response_seconds"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace("seconds", "", regex=False)
    .str.strip()
    .replace("", None)
    .astype(float)
)

In [13]:
# Merge the two datasets
merged = pd.merge(
    calls[["borough", "num_incidents", "response_seconds"]],
    pop[["borough", "population"]],
    on="borough",
    how="left"
)

merged.head()

,borough,num_incidents,response_seconds,population
0,Brooklyn,13913.0,620.29,2648452.0
1,Bronx,11423.0,894.03,1446788.0
2,Manhattan,11883.0,786.83,1638281.0
3,Queens,10060.0,694.58,2330295.0
4,Staten Island,2178.0,589.96,487155.0


In [14]:
# Aggregate by borough
per_boro = (
    merged
    .groupby("borough", as_index=False)
    .agg(
        total_calls=("num_incidents","sum"),
        avg_response=("response_seconds","mean"),
        population=("population","first")
    )
)

# Per 1,000 residents metric
per_boro["calls_per_1000"] = per_boro["total_calls"] / per_boro["population"] * 1000

per_boro


,borough,total_calls,avg_response,population,calls_per_1000
0,Bronx,8339844.0,357.250334,1446788.0,5764.385660
1,Brooklyn,10461352.0,313.855121,2648452.0,3949.987389
2,Manhattan,8530719.0,335.943481,1638281.0,5207.115873
3,Queens,7309853.0,333.643987,2330295.0,3136.878807
4,Staten Island,1682440.0,311.313601,487155.0,3453.603063
5,Unspecified,22342.0,511.779889,NaN,NaN


In [17]:
fig = px.scatter(
    per_boro,
    x="calls_per_1000",
    y="avg_response",
    text="borough",
    labels={
        "calls_per_1000": "911 Calls per 1,000 Residents",
        "avg_response": "Average Response Time (seconds)"
    },
    title="Relationship Between 911 Demand and Response Time Across NYC Boroughs"
)

fig.update_traces(textposition="top center")
fig.show()

Takeaways
Boroughs with higher per-capita 911 incident rates do not necessarily experience slower response times.
Response efficiency may depend more on resource allocation, traffic congestion, and borough geography.
Staten Island shows low incident rates but does not have proportionally faster response times, suggesting structural constraints.
This analysis demonstrates the importance of normalizing total incidents by population to avoid misleading comparisons.